## 97. Interleaving String
- Description:
  <blockquote>
    Given strings `s1`, `s2`, and `s3`, find whether `s3` is formed by an **interleaving** of `s1` and `s2`.
     
    An **interleaving** of two strings `s` and `t` is a configuration where `s` and `t` are divided into `n` and `m`substringsrespectively, such that:
     
    - `s = s1+ s2+ ... + sn`
    - `t = t1+ t2+ ... + tm`
    - `|n - m| <= 1`
    - The **interleaving** is `s1+ t1+ s2+ t2+ s3+ t3+ ...` or `t1+ s1+ t2+ s2+ t3+ s3+ ...`
     
    **Note:** `a + b` is the concatenation of strings `a` and `b`.
     
    **Example 1:**
    ![Image](https://assets.leetcode.com/uploads/2020/09/02/interleave.jpg)
     
    **Input:** s1 = "aabcc", s2 = "dbbca", s3 = "aadbbcbcac"
    **Output:** true
    **Explanation:** One way to obtain s3 is:
    Split s1 into s1 = "aa" + "bc" + "c", and s2 into s2 = "dbbc" + "a".
    Interleaving the two splits, we get "aa" + "dbbc" + "bc" + "a" + "c" = "aadbbcbcac".
    Since s3 can be obtained by interleaving s1 and s2, we return true.
     
    **Example 2:**
    **Input:** s1 = "aabcc", s2 = "dbbca", s3 = "aadbbbaccc"
    **Output:** false
    **Explanation:** Notice how it is impossible to interleave s2 with any other string to obtain s3.
     
    **Example 3:**
    **Input:** s1 = "", s2 = "", s3 = ""
    **Output:** true
     
    **Constraints:**
     
    - `0 <= s1.length, s2.length <= 100`
    - `0 <= s3.length <= 200`
    - `s1`, `s2`, and `s3` consist of lowercase English letters.
     
    **Follow up:** Could you solve it using only `O(s2.length)` additional memory space?
  </blockquote>

- URL: Problem_URL

- Topics: [Problem_topic](https://leetcode.com/problems/interleaving-string/description/)

- Difficulty: Medium / Hard

- Resources: example_resource_URL

### Solution 1, Recursion with memoization

Time complexity : O(2m+n). m is the length of s1 and n is the length of s2.
Space complexity : O(m+n). The size of stack for recursive calls can go upto m+n.

In [ ]:
class Solution:
    def is_Interleave(self, s1: str, i: int, s2: str, j: int, s3: str, k: int, memo: list) -> bool:
        if i == len(s1):
            return s2[j:] == s3[k:]

        if j == len(s2):
            return s1[i:] == s3[k:]

        if memo[i][j] >= 0:
            return memo[i][j] == 1
        
        ans = False

        if (
            s3[k] == s1[i]
            and self.is_Interleave(s1, i + 1, s2, j, s3, k + 1, memo)
            or s3[k] == s2[j]
            and self.is_Interleave(s1, i, s2, j + 1, s3, k + 1, memo)
        ):
            ans = True
        
        memo[i][j] = 1 if ans else 0
        return ans

    def isInterleave(self, s1: str, s2: str, s3: str) -> bool:
        if len(s1) + len(s2) != len(s3):
            return False

        memo = [[-1] * len(s2) for _ in range(len(s1))]
        return self.is_Interleave(s1, 0, s2, 0, s3, 0, memo)

### Solution 2, 2D Dynamic Programming

Time complexity : O(m⋅n). dp array of size m∗n is filled.
Space complexity : O(m⋅n). 2D dp of size (m+1)∗(n+1) is required. m and n are the lengths of strings s1 and s2 respectively.

In [ ]:
class Solution:
    def isInterleave(self, s1: str, s2: str, s3: str) -> bool:
        s1_len = len(s1)
        s2_len = len(s2)
        s3_len = len(s3)

        if s3_len != s1_len+s2_len:
            return False
        
        # dp[i][j] represents: "Can we form the first i + j characters of s3 using the first i characters of s1 and first j characters of s2?"
        dp = [[False] * (s2_len + 1) for _ in range(s1_len + 1)]

        for i in range(s1_len+1):
            for j in range(s2_len+1):
                if i == 0 and j == 0:
                    dp[i][j] = True
                elif i == 0:
                    dp[i][j] = dp[i][j-1] and s2[j-1] == s3[i+j-1]
                elif j == 0:
                    dp[i][j] = dp[i-1][j] and s1[i-1] == s3[i+j-1]
                else:
                    dp[i][j] = ((dp[i][j-1] and s2[j-1] == s3[i+j-1]) 
                    or (dp[i-1][j] and s1[i-1] == s3[i+j-1]))
        
        return dp[s1_len][s2_len]

### Solution 3, Space Optimized 1D Dynamic Programming
Time complexity : O(m⋅n). dp array of size n is filled m times.
Space complexity : O(n). n is the length of the string s1.

In [ ]:
class Solution:
    def isInterleave(self, s1: str, s2: str, s3: str) -> bool:
        if len(s3) != len(s1) + len(s2):
            return False

        # dp[j] represents: Can we form the first i+j characters of s3 using the first i characters of s1 and first j characters of s2?
        dp = [False] * (len(s2) + 1)

        for i in range(len(s1) + 1):
            for j in range(len(s2) + 1):
                if i == 0 and j == 0:
                    dp[j] = True
                elif i == 0:
                    dp[j] = dp[j - 1] and s2[j - 1] == s3[i + j - 1]
                elif j == 0:
                    dp[j] = dp[j] and s1[i - 1] == s3[i + j - 1]
                else:
                    # dp[j] (before update) = value from previous row at column j → represents state (i-1, j)
                    # dp[j-1] = value from current row at column j-1 → represents state (i, j-1)
                    dp[j] = (dp[j] and s1[i - 1] == s3[i + j - 1]) or (
                        dp[j - 1] and s2[j - 1] == s3[i + j - 1]
                    )
        
        return dp[len(s2)]